In [1]:
import os
import sys
import glob
import json
import numpy as np
import pandas as pd


In [18]:
def get_layer(dfi, n_classes=[]):
    name = dfi.layer_name.lower()
    num = ''.join(_ for _ in name if _.isdigit())
    if 'flatten' in name:
        layer = 'flatten'
    elif 'reshape' in name:
        layer = None
    elif 'dropout' in name:
        layer = f'dropout'
    elif 'fc' in name:
        if 'intermediate' in name:
            num = '0'
        else:
            num = ''
        layer = f'fc{num}'
        units = dfi.args.get('units', n_classes)
        if not isinstance(units, list):
            units = [units]
        layer += ' {}'.format(units)
        if 'norm' in name:
            layer = f'norm_fc{num}'
#             if 'batch' in dfi.layer_type.lower():
#                 layer = 'b' + layer 
#             elif 'layer' in dfi.layer_type.lower():
#                 layer = 'l' + layer
        if 'relu' in name:
            layer = f'relu_fc{num}'
    elif 'conv' in name:
        layer = f'conv{num}'
        layer += ' {}'.format(dfi.args['kernel_size'] + [dfi.args['filters']])
    elif 'pool' in name:
        layer = f'pool{num}'
        if dfi.args.get('strides', None) is not None:
            layer += ' {}'.format(dfi.args['strides'])
        else:
            layer += ' {}'.format(dfi.args['pool_size'])
        if 'max' in dfi.layer_type.lower():
            layer = 'm' + layer 
        else:
            layer = 'h' + layer 
    elif 'relu' in name:
        layer = f'relu{num}'
    elif 'norm' in name:
        layer = f'norm{num}'
        if 'batch' in dfi.layer_type.lower():
            layer = 'b' + layer 
        elif 'layer' in dfi.layer_type.lower():
            layer = 'l' + layer 
        if 'input' in name:
            layer = 'input_' + layer
    else:
        layer = None
    return layer


list_regex_fn_arch = [
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/IHC3000Hz_anf384H160M096L/arch*/arch.json',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/IHC3000Hz_anf192H080M048L/arch*/arch.json',
    '/om2/user/msaddler/tfauditoryutil/saved_models/augmented_2022JAN/taskSW/IHC3000Hz_anf384H160M096L/arch*/arch.json',
]

list_df = []
for regex_fn_arch in list_regex_fn_arch:
    list_fn_arch = glob.glob(regex_fn_arch)
    for itr_arch, fn_arch in enumerate(sorted(list_fn_arch)):
        tag_arch = os.path.basename(os.path.dirname(fn_arch))
        df = pd.read_json(fn_arch)
        df['tag_arch'] = tag_arch
        df['itr_arch'] = itr_arch
        fn_config = fn_arch.replace('arch.json', 'config.json')
        with open(fn_config, 'r') as f:
            CONFIG = json.load(f)
            n_classes = [v for k, v in CONFIG['n_classes_dict'].items()]
        df['layer_name'] = df['args'].map(lambda _: _['name'])
        df['layer'] = df.apply(lambda _: get_layer(_, n_classes=n_classes), axis=1)
        df = df[~df['layer'].isna()].reset_index(drop=True)
        df = df.drop(columns=['args', 'layer_type', 'layer_name'])
        list_df.append(df)
df = pd.concat(list_df)

table_df = {}
for name, sub_df in df.groupby('tag_arch'):
    table_df[name] = sub_df['layer']
table_df = pd.DataFrame(table_df).fillna('')
# table_df.to_csv('/om2/user/msaddler/tfauditoryutil/paper_phaselocknet/data/model_arch.csv', index=False)
table_df


,arch0_0000,arch0_0001,arch0_0002,arch0_0004,arch0_0006,arch0_0007,arch0_0008,arch0_0009,arch0_0016,arch0_0017,...,arch_0083,arch_0154,arch_0190,arch_0191,arch_0286,arch_0288,arch_0302,arch_0335,arch_0338,arch_0346
0,input_lnorm,input_lnorm,input_lnorm,input_lnorm,input_lnorm,input_lnorm,input_lnorm,input_lnorm,input_lnorm,input_lnorm,...,"conv0 [2, 82, 32]","conv0 [1, 70, 64]","conv0 [2, 97, 32]","conv0 [2, 83, 32]","conv0 [1, 180, 64]","conv0 [3, 77, 64]","conv0 [1, 250, 32]","conv0 [1, 71, 32]","conv0 [2, 110, 64]","conv0 [3, 53, 32]"
1,"conv0 [2, 42, 32]","conv0 [1, 84, 32]","conv0 [4, 21, 32]","conv0 [2, 42, 32]","conv0 [2, 42, 32]","conv0 [2, 42, 32]","conv0 [2, 42, 32]","conv0 [2, 42, 32]","conv0 [2, 42, 32]","conv0 [2, 42, 32]",...,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0
2,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,...,"hpool0 [1, 2]","hpool0 [1, 5]","hpool0 [1, 6]","hpool0 [1, 2]","hpool0 [2, 6]","hpool0 [1, 2]","hpool0 [1, 5]","hpool0 [1, 1]","hpool0 [3, 2]","hpool0 [1, 2]"
3,"hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]","hpool0 [2, 4]",...,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0
4,lnorm0,lnorm0,lnorm0,lnorm0,lnorm0,lnorm0,lnorm0,lnorm0,lnorm0,lnorm0,...,"conv1 [1, 162, 64]","conv1 [7, 21, 128]","conv1 [5, 11, 64]","conv1 [1, 164, 64]","conv1 [2, 37, 128]","conv1 [1, 193, 128]","conv1 [19, 11, 64]","conv1 [1, 114, 32]","conv1 [1, 126, 128]","conv1 [1, 60, 64]"
5,"conv1 [2, 18, 64]","conv1 [2, 18, 64]","conv1 [2, 18, 64]","conv1 [4, 9, 64]","conv1 [2, 18, 64]","conv1 [2, 18, 64]","conv1 [2, 18, 64]","conv1 [2, 18, 64]","conv1 [2, 18, 64]","conv1 [2, 18, 64]",...,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1
6,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,...,"hpool1 [2, 2]","hpool1 [4, 3]","hpool1 [1, 1]","hpool1 [3, 7]","hpool1 [1, 1]","hpool1 [4, 3]","hpool1 [1, 7]","hpool1 [1, 3]","hpool1 [3, 3]","hpool1 [2, 4]"
7,"hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]",...,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1
8,lnorm1,lnorm1,lnorm1,lnorm1,lnorm1,lnorm1,lnorm1,lnorm1,lnorm1,lnorm1,...,"conv2 [1, 72, 128]","conv2 [4, 26, 256]","conv2 [1, 56, 128]","conv2 [5, 9, 128]","conv2 [15, 10, 128]","conv2 [8, 10, 128]","conv2 [12, 9, 128]","conv2 [1, 86, 64]","conv2 [4, 30, 256]","conv2 [3, 46, 128]"
9,"conv2 [6, 6, 128]","conv2 [6, 6, 128]","conv2 [6, 6, 128]","conv2 [6, 6, 128]","conv2 [12, 3, 128]","conv2 [6, 6, 128]","conv2 [6, 6, 128]","conv2 [6, 6, 128]","conv2 [6, 6, 128]","conv2 [6, 6, 128]",...,relu2,relu2,relu2,relu2,relu2,relu2,relu2,relu2,relu2,relu2
